In [ ]:
# Research Questions: Did the COVID-19 permanently affect the demand for small homes?

In [ ]:
### While the NDSS provides a reasonable standard for the study of housing, houses are classified according to the number of bedrooms, number of storeys and total floor area. However, House Price per Square Metre in England and Wales exclude the number of bedrooms and the number of storeys. To solve this lack of housing information, reference was made to previous studies and additional information was added. 

In [1]:
import pandas as pd
import glob

In [ ]:
### Combined House Prices per Square Meter in England and Wales 2022 data ###

In [ ]:
# Path to the folder containing the CSV files
path = './Data housing all' 

# Using glob to match all .csv files in the folder
all_files = glob.glob(path + "/*.csv")

# Creating an empty list to hold the DataFrames
li = []

# Reading each CSV file and appending it to the list
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concatenating all the DataFrames in the list
frame = pd.concat(li, axis=0, ignore_index=True)

# Saving the concatenated DataFrame to a new CSV file
frame.to_csv("SH_1_combined_three periods.csv", index=False)

In [7]:
### Cleaning Data ###

In [33]:
home2022 = pd.read_csv("./SH_1_combined_three periods.csv")

# Display the count of missing values in each column
missing_values_count = home2022.isnull().sum()
print(missing_values_count)


priceper                          676
year                                0
dateoftransfer                      0
propertytype                        0
duration                            0
price                               0
postcode                            0
lad21cd                             0
transactionid                       0
id                                  0
tfarea                              0
numberrooms                    419102
classt                              0
CURRENT_ENERGY_EFFICIENCY           0
POTENTIAL_ENERGY_EFFICIENCY         0
CONSTRUCTION_AGE_BAND           38495
dtype: int64


In [38]:
# Drop rows with any missing values
home2022_cleaned = home2022.dropna()

# Check After Cleaning
remaining_missing_values = home2022_cleaned.isnull().sum()
print(remaining_missing_values)

# Export data as CSV file
# home2022_cleaned.to_csv('./SH_2_combined_three periods_cleaned.csv', index=False)

priceper                       0
year                           0
dateoftransfer                 0
propertytype                   0
duration                       0
price                          0
postcode                       0
lad21cd                        0
transactionid                  0
id                             0
tfarea                         0
numberrooms                    0
classt                         0
CURRENT_ENERGY_EFFICIENCY      0
POTENTIAL_ENERGY_EFFICIENCY    0
CONSTRUCTION_AGE_BAND          0
dtype: int64


In [ ]:
###  Additional information on the house: number of storeys  ###

In [28]:

# Add storey numbers for each type of house
property_type_mapping = {'T': 2, 'D': 2, 'S': 2, 'F': 1}
home2022_cleaned.loc[:, 'storey'] = home2022_cleaned['propertytype'].map(property_type_mapping)

# Display the modified DataFrame
print(home2022_cleaned)


            priceper  year dateoftransfer propertytype duration    price  \
0        3842.696629  2020     2020-01-23            T        F   342000   
1        4550.561798  2022     2022-01-05            T        F   405000   
2        4244.031830  2021     2021-11-05            T        F   320000   
3        4258.064516  2019     2019-12-20            D        F   264000   
4        4602.272727  2021     2021-06-29            D        F   405000   
...              ...   ...            ...          ...      ...      ...   
2599266  3693.181818  2019     2019-02-08            D        F   975000   
2599267  4328.358209  2021     2021-09-09            D        F  1450000   
2599268  4757.575758  2020     2020-03-11            S        F   785000   
2599269  6400.000000  2021     2021-10-15            D        F   960000   
2599270  4610.389610  2019     2019-08-08            D        F   710000   

         postcode    lad21cd                           transactionid  \
0        BN14 9

/tmp/ipykernel_12573/3419952148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home2022_cleaned.loc[:, 'storey'] = home2022_cleaned['propertytype'].map(property_type_mapping)


In [ ]:
### After adding the key information, the subtypes of detached, semi-detached, terraced and flat were classified according to the NDSS: "Small Home" and " Comfortable Home"

In [ ]:
###  Screening out small homes  ###

In [72]:
# Small Homes

# Filter the data based on criteria: Detached
smallhomes_detached = home2022_cleaned[(home2022_cleaned['propertytype'] == 'D') & 
                     (home2022_cleaned['numberrooms'] >= 4) & 
                     (home2022_cleaned['tfarea'] >= 97) & (home2022_cleaned['tfarea'] <= 124)]


# Filter the data based on criteria: Semi-Detached
smallhomes_semidetached = home2022_cleaned[(home2022_cleaned['propertytype'] == 'S') & 
                     (home2022_cleaned['numberrooms'] >= 3) & 
                     (home2022_cleaned['tfarea'] >= 84) & (home2022_cleaned['tfarea'] <= 102)]


# Filter the data based on criteria: Terraced
smallhomes_terraced = home2022_cleaned[(home2022_cleaned['propertytype'] == 'T') & 
                     (home2022_cleaned['numberrooms'] >= 2) & 
                     (home2022_cleaned['tfarea'] >= 70) & (home2022_cleaned['tfarea'] <= 79)]


# Filter the data based on criteria: Flat or Maisonettes
# Define the conditions for filtering
condition_1 = (home2022_cleaned['propertytype'] == 'F') & (home2022_cleaned['tfarea'] >= 39) & (home2022_cleaned['tfarea'] <= 50) & (home2022_cleaned['numberrooms'] >= 1)
condition_2 = (home2022_cleaned['propertytype'] == 'F') & (home2022_cleaned['tfarea'] >= 61) & (home2022_cleaned['tfarea'] <= 70) & (home2022_cleaned['numberrooms'] >= 2)

# Apply the conditions and concatenate the results
smallhomes_flat = pd.concat([home2022_cleaned[condition_1], home2022_cleaned[condition_2]])



# unified filtered data
smallhomes_all = pd.concat([smallhomes_detached, smallhomes_semidetached, smallhomes_terraced, smallhomes_flat])

# print(smallhomes_all)

# export data
# smallhomes_all.to_csv('./SH_3_Small Homes_all_housing type.csv', index=False)


In [ ]:
###  Screening out reasonable and comfortable homes  ###

In [83]:
# Comfortable Homes

# Filter the data based on criteria   
# Detached
comfortablehomes_detached = home2022_cleaned[(home2022_cleaned['propertytype'] == 'D') & 
                     (home2022_cleaned['numberrooms'] >= 4) & 
                     (home2022_cleaned['tfarea'] > 124)]


# Filter the data based on criteria   
# Semi-Detached
comfortablehomes_semidetached = home2022_cleaned[(home2022_cleaned['propertytype'] == 'S') & 
                     (home2022_cleaned['numberrooms'] >= 3) & 
                     (home2022_cleaned['tfarea'] > 102)]


# Filter the data based on criteria   
# Terraced
comfortablehomes_terraced = home2022_cleaned[(home2022_cleaned['propertytype'] == 'T') & 
                     (home2022_cleaned['numberrooms'] >= 2) & 
                     (home2022_cleaned['tfarea'] > 79)]


# Filter the data based on criteria   
# Flat or Maisonettes
condition_3 = (home2022_cleaned['propertytype'] == 'F') & (home2022_cleaned['tfarea'] > 50) & (home2022_cleaned['numberrooms'] >= 1)
condition_4 = (home2022_cleaned['propertytype'] == 'F') & (home2022_cleaned['tfarea'] > 70) & (home2022_cleaned['numberrooms'] >= 2)

# Apply the conditions and concatenate the results
comfortablehomes_flat = pd.concat([home2022_cleaned[condition_3], home2022_cleaned[condition_4]])


# unified filtered data
comfortablehomes_all = pd.concat([comfortablehomes_detached, comfortablehomes_semidetached, comfortablehomes_terraced, comfortablehomes_flat])

# print(comfortablehomes_all)

# export data
# comfortablehomes_all.to_csv('./SH_4_Comfortable Homes_all_housing type.csv', index=False)
